In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%tensorflow_version 1.x

# get the Anaconda file 
! wget -c https://repo.anaconda.com/archive/Anaconda3-2019.10-Linux-x86_64.sh
! chmod +x Anaconda3-2019.10-Linux-x86_64.sh
! bash ./Anaconda3-2019.10-Linux-x86_64.sh -b -f -p /usr/local

! time conda install -y -c deepchem -c rdkit -c conda-forge -c omnia deepchem-gpu=2.3.0

import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')
import deepchem as dc

# install mordred, bravado and molvs
! time conda install -c conda-forge -y mordred bravado molvs

# get the Install AMPL_GPU_test.sh
!wget https://raw.githubusercontent.com/ravichas/AMPL-Tutorial/master/config/install_AMPL_GPU_test.sh

# run the script to install AMPL
! chmod u+x install_AMPL_GPU_test.sh
! ./install_AMPL_GPU_test.sh


TensorFlow 1.x selected.
--2021-04-01 15:52:32--  https://repo.anaconda.com/archive/Anaconda3-2019.10-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 530308481 (506M) [application/x-sh]
Saving to: ‘Anaconda3-2019.10-Linux-x86_64.sh’

Anaconda3-2019.10-L 100%[===================>] 505.74M   164MB/s    in 3.2s    

2021-04-01 15:52:35 (159 MB/s) - ‘Anaconda3-2019.10-Linux-x86_64.sh’ saved [530308481/530308481]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _ipyw_jlab_nb_ext_conf==0.1.0=py37_0
    - _libgcc_mutex==0.1=main
    - alabaster==0.7.12=py37_0
    - anaconda-client==1.7.2=py37_0
    - a

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.


InvalidVersionSpec: Invalid version '4.19.112+': empty version component


real	0m10.866s
user	0m7.877s
sys	0m1.878s
--2021-04-01 16:01:05--  https://raw.githubusercontent.com/ravichas/AMPL-Tutorial/master/config/install_AMPL_GPU_test.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1100 (1.1K) [text/plain]
Saving to: ‘install_AMPL_GPU_test.sh’

install_AMPL_GPU_te 100%[===================>]   1.07

In [8]:
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
# manipulate data
import pandas as pd

import json
import requests
import io
import seaborn as sns
import matplotlib.pyplot as plt

import matplotlib_venn as mpv
from scipy.stats import pearsonr

# set up visualization parameters
sns.set_context("paper")
sns.set_style("whitegrid")
sns.set_palette("Set2")
pal = sns.color_palette()
plt.rcParams['figure.figsize'] = [10,10]

pd.set_option('display.max_columns',(90))
pd.set_option('display.max_rows',(20))

In [13]:
# Import AMPL libraries
# 
import atomsci.ddm.utils.data_curation_functions as dcf
import atomsci.ddm.utils.curate_data as curate_data
import atomsci.ddm.pipeline.diversity_plots as dp
import atomsci.ddm.pipeline.chem_diversity as cd
# Additional python libraries
import pandas as pd
import numpy as np
import getpass, os
import io

In [14]:
from google.colab import files

In [15]:
uploaded = files.upload()

Saving ChEMBL.csv to ChEMBL.csv


In [18]:
chembl = pd.read_csv(io.StringIO(uploaded['ChEMBL.csv'].decode('ISO-8859-1')))

In [24]:
chembl.columns = chembl.columns.str.replace(' ','_')

In [25]:
chembl['Standard_Value_nM'] = np.where(chembl.Standard_Units == 'uM',
                                           chembl.Standard_Value * 1000,
                                           chembl.Standard_Value)

In [26]:
chembl = chembl.replace('None',np.nan)

In [28]:
num_cols = ['Molecule_Max_Phase',
       'Molecular_Weight', '#RO5_Violations', 'AlogP', 'Standard_Value_nM',
       'pChEMBL_Value', 'Ligand_Efficiency_BEI', 'Ligand_Efficiency_LE',
       'Ligand_Efficiency_LLE', 'Ligand_Efficiency_SEI']
for col in num_cols:
  chembl[col] = chembl[col].astype(float)

In [30]:
uploaded = files.upload()

Saving chembl_smile.txt to chembl_smile.txt


In [31]:
chembl_smiles = pd.read_csv('chembl_smile.txt',sep='\t',header=None)
chembl_smiles.rename(columns={0:'ChEMBL_ID',1:'SMILES'})
#This step is trivial, since ChEMBL dataset has the 'smile' column already.

,0,1
0,CHEMBL4539943,C1C[C@H]([C@@H]([C@H]1CN2C(=O)C3=CC=CC=C3N=N2)...
1,CHEMBL4163587,COC1=C(C=C2C(=C1)C(=NC(=N2)N3CCC[C@@H](C3)NC4=...
2,CHEMBL234679,CC(C)OC1=CC=CC=C1N2CCN(CC2)C3CCC(CC3)NS(=O)(=O...
3,CHEMBL15245,COC(=O)[C@H]1[C@H](CC[C@@H]2[C@@H]1C[C@H]3C4=C...
4,CHEMBL293980,C1CCC2(C1)CC(=O)N(C(=O)C2)CCN3CCN(CC3)C4=C(C=C...


In [32]:
chembl['base_rdkit_smiles'] = chembl.Smiles.apply(curate_data.base_smiles_from_smiles)
chembl.replace('',np.nan)
chembl=chembl[~chembl['base_rdkit_smiles'].isna()]

In [33]:
chembl['PChEMBL_Value_nM'] = -np.log10(chembl['Standard_Value_nM']/1000000000)

In [34]:
chembl.head()

,Molecule_ChEMBL_ID,Molecule_Name,Molecule_Max_Phase,Molecular_Weight,#RO5_Violations,AlogP,Compound_Key,Smiles,Standard_Type,Standard_Relation,Standard_Value,Standard_Units,pChEMBL_Value,Data_Validity_Comment,Comment,Uo_Units,Ligand_Efficiency_BEI,Ligand_Efficiency_LE,Ligand_Efficiency_LLE,Ligand_Efficiency_SEI,Potential_Duplicate,Assay_ChEMBL_ID,Assay_Description,Assay_Type,BAO_Format_ID,BAO_Label,Assay_Organism,Assay_Tissue_ChEMBL_ID,Assay_Tissue_Name,Assay_Cell_Type,Assay_Subcellular_Fraction,Assay_Parameters,Assay_Variant_Accession,Assay_Variant_Mutation,Target_ChEMBL_ID,Target_Name,Target_Organism,Target_Type,Document_ChEMBL_ID,Source_ID,Source_Description,Document_Journal,Document_Year,Cell_ChEMBL_ID,Properties,Standard_Value_nM,base_rdkit_smiles,PChEMBL_Value_nM
0,CHEMBL4539943,NaN,0.0,505.57,1.0,3.60,NaN,O=C(c1ccc(OCCC2CCOCC2)cc1)[C@@H]1CC[C@H](Cn2nn...,Inhibition,'=',6.2900,%,NaN,NaN,Not Active,UO_0000187,NaN,NaN,NaN,NaN,False,CHEMBL4507829,GPCRScan assay: inhibition of Alpha1B,B,BAO_0000357,single protein format,Homo sapiens,NaN,NaN,NaN,NaN,Parental cells: stable HEK | PMID: 7780636/168...,NaN,NaN,CHEMBL232,Alpha-1b adrenergic receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL4507264,54.0,Donated Chemical Probes - SGC Frankfurt,NaN,NaN,NaN,Compound concentration None 10.0 uM,6.2900,O=C(c1ccc(OCCC2CCOCC2)cc1)[C@@H]1CC[C@H](Cn2nn...,8.201349
1,CHEMBL4163587,NaN,0.0,459.51,0.0,2.75,10,COc1cc2nc(N3CCC[C@H](NC4=CC(=O)c5ccccc5C4=O)C3...,Ki,'=',0.5248,nM,9.28,NaN,NaN,UO_0000065,20.20,0.37,6.53,7.75,False,CHEMBL4139219,Displacement of [3H]prazosin from human alpha1...,B,BAO_0000219,cell-based format,Homo sapiens,NaN,NaN,CHO,NaN,NaN,NaN,NaN,CHEMBL232,Alpha-1b adrenergic receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL4138091,1.0,Scientific Literature,Eur J Med Chem,2017.0,NaN,NaN,0.5248,COc1cc2nc(N3CCC[C@H](NC4=CC(=O)c5ccccc5C4=O)C3...,9.280006
2,CHEMBL234679,NaN,0.0,482.65,0.0,3.76,6cis,CC(C)Oc1ccccc1N1CCN([C@H]2CC[C@@H](NS(=O)(=O)c...,Ki,'=',217.0000,nM,6.66,NaN,NaN,UO_0000065,13.81,0.27,2.90,7.78,False,CHEMBL894517,Binding affinity to human cloned adrenergic al...,B,BAO_0000357,single protein format,Homo sapiens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHEMBL232,Alpha-1b adrenergic receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL1148021,1.0,Scientific Literature,Bioorg. Med. Chem. Lett.,2007.0,NaN,NaN,217.0000,CC(C)Oc1ccccc1N1CCN([C@H]2CC[C@@H](NS(=O)(=O)c...,6.663540
3,CHEMBL15245,YOHIMBINE,4.0,354.45,0.0,2.65,Yohimbine,COC(=O)[C@H]1[C@@H](O)CC[C@H]2CN3CCc4c([nH]c5c...,Ki,NaN,NaN,NaN,NaN,NaN,Not Determined,NaN,NaN,NaN,NaN,NaN,False,CHEMBL4380086,Displacement of [3H]prazosin from human recomb...,B,BAO_0000219,cell-based format,Homo sapiens,NaN,NaN,CHO,NaN,NaN,NaN,NaN,CHEMBL232,Alpha-1b adrenergic receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL4376854,1.0,Scientific Literature,J Med Chem,2020.0,CHEMBL3308072,NaN,NaN,COC(=O)[C@H]1[C@@H](O)CC[C@H]2CN3CCc4c([nH]c5c...,NaN
4,CHEMBL293980,NaN,0.0,424.37,0.0,3.82,70,O=C1CC2(CCCC2)CC(=O)N1CCN1CCN(c2ccc(Cl)cc2Cl)CC1,Ki,NaN,NaN,NaN,NaN,NaN,Not Determined,NaN,NaN,NaN,NaN,NaN,False,CHEMBL649208,Binding affinity constant against alpha-1B adr...,B,BAO_0000221,tissue-based format,Cavia porcellus,CHEMBL3559722,Spleen,NaN,NaN,NaN,NaN,NaN,CHEMBL232,Alpha-1b adrenergic receptor,Homo sapiens,SINGLE PROTEIN,CHEMBL1149137,1.0,Scientific Literature,J. Med. Chem.,2004.0,NaN,NaN,NaN,O=C1CC2(CCCC2)CC(=O)N1CCN1CCN(c2ccc(Cl)cc2Cl)CC1,NaN


In [35]:
thresh=-np.log10(100/1000000000)

In [39]:
chembl['pRelation'] = chembl.Standard_Relation
chembl['pRelation'] = np.where(chembl.Standard_Relation == "'<'", ">", chembl.pRelation)
chembl['pRelation'] = np.where(chembl.Standard_Relation == "'>'", "<", chembl.pRelation)
chembl['pRelation'] = np.where(chembl.Standard_Relation == "'='", "=", chembl.pRelation)

In [51]:
# select pIC50 values
vals = ['EC50', 'IC50', 'pIC50']
chembl=chembl[chembl.Standard_Type.isin(vals)]
chembl=chembl.dropna(axis=0, subset = ['PChEMBL_Value_nM','base_rdkit_smiles'])

chembl_cur = curate_data.aggregate_assay_data(chembl, value_col = 'PChEMBL_Value_nM', 
                                              output_value_col = 'pXC50',
                                              id_col = 'Molecule_ChEMBL_ID', 
                                              smiles_col = 'base_rdkit_smiles',
                                              relation_col = 'pRelation',
                                              active_thresh=7)
chembl_cur.head(20)

0 entries in input table are missing SMILES strings
343 unique SMILES strings are reduced to 343 unique base SMILES strings


,compound_id,base_rdkit_smiles,relation,pXC50,active
0,CHEMBL137487,N=C(N)c1ccc(CNC(=O)Cc2c(-c3cccc(N)c3)ccc(NCc3c...,,5.000000,0
1,CHEMBL17819,Cc1nc(-c2ccccc2NCC2=NCCN2)n(C)n1,>,5.300000,0
2,CHEMBL3810005,CC(Cc1cc2c(c(C(N)=O)c1)N(CCCO)CC2)NCCOc1cc(Cl)...,,5.918653,0
3,CHEMBL18303,CC(C)n1nccc1-c1ccccc1NCC1=NCCN1,>,4.000000,0
4,CHEMBL257976,CNC(=O)c1cccc2c1CCC2c1ncc[nH]1,<,5.000000,0
5,CHEMBL250699,COc1cc(NC(=O)c2ccc(-c3ccc(Cl)cc3)o2)cc(OC)c1,<,5.000000,0
6,CHEMBL306792,CSc1cc(F)ccc1NCC1=NCCN1,,8.219971,1
7,CHEMBL134792,N=C(N)c1ccc(CNC(=O)Cc2c(F)ccc(NCCc3ccccc3)c2F)cc1,,5.806875,0
8,CHEMBL3742103,Cc1ccc(Cl)cc1N1CCN(CCc2ccc(CN3C(=O)c4ccccc4C3=...,,7.500038,1
9,CHEMBL430717,COc1cc2nc(N3CCN(C(=O)COc4c(OC)cccc4C(C)C)CC3)n...,,8.920819,1


In [52]:
chembl_cur.to_csv('chembl_ready.csv')

In [54]:
from atomsci.ddm.pipeline import model_pipeline as mp
from atomsci.ddm.pipeline import parameter_parser as parse
from atomsci.ddm.pipeline import perf_data

from hyperopt import fmin, tpe, hp, Trials, STATUS_OK

In [55]:
train_file = "/content/chembl_ready.csv"
response_col = "pXC50"
compound_id = "compound_id"
smiles_col = "base_rdkit_smiles"

params = {
        "system": "LC",
        "lc_account": 'None',
        "datastore": "False",
        "save_results": "False",
        "data_owner": "username",
        "prediction_type": "regression",
        "dataset_key": train_file,
        "id_col": compound_id,
        "smiles_col": smiles_col,
        "response_cols": response_col,
        "previously_split": "False",
        "split_only": "True",
        "featurizer": "ecfp",
        "model_type": "RF",
        "verbose": "True",
        "transformers": "True",
        'max_epochs': '70',
        "rerun": "False",
        "result_dir": "split"
    }

ampl_param = parse.wrapper(params)
pl = mp.ModelPipeline(ampl_param)
pl.train_model()

2021-04-01 17:03:01,260 Splitting data by scaffold


number of features: 1024


2021-04-01 17:03:01,674 Dataset split table saved to /content/chembl_ready_train_valid_test_scaffold_ffaf82ac-4178-4811-8952-0e1c9d257e8e.csv
